# AnkiWeb Sync

Sync cards to AnkiWeb automatically after adding them via AnkiConnect.

**Requirements**:
- Anki must be running (headless via Xvfb in Docker)
- AnkiConnect plugin installed
- AnkiWeb credentials configured via environment variables

In [ ]:
#| default_exp ankiweb

In [ ]:
#| export
import os
import time
from typing import Optional

In [ ]:
#| export
class SyncError(Exception):
    """Raised when AnkiWeb sync fails after all retries."""
    pass

## Configuration

In [ ]:
#| export
def configure_ankiweb(
    username: Optional[str] = None,  # AnkiWeb username (default: ANKIWEB_USERNAME env var)
    password: Optional[str] = None   # AnkiWeb password (default: ANKIWEB_PASSWORD env var)
) -> dict:
    """Configure AnkiWeb credentials for sync.
    
    Reads credentials from environment variables if not provided.
    
    Returns:
        dict: Credential configuration
    
    Raises:
        ValueError: If credentials not found
    """
    username = username or os.getenv("ANKIWEB_USERNAME")
    password = password or os.getenv("ANKIWEB_PASSWORD")
    
    if not username or not password:
        raise ValueError(
            "AnkiWeb credentials not found. Set ANKIWEB_USERNAME and ANKIWEB_PASSWORD "
            "environment variables or pass them as arguments."
        )
    
    # TODO: Store credentials in Anki's preferences
    # This requires investigation of AnkiConnect's capabilities
    return {"username": username, "password": "***"}

## Sync Functions

In [ ]:
#| export
def sync_to_ankiweb(max_retries: int = 3) -> None:
    """Sync local collection to AnkiWeb with retry logic.
    
    Args:
        max_retries: Maximum number of retry attempts (default: 3)
    
    Raises:
        SyncError: If sync fails after all retries
    
    Notes:
        - Uses exponential backoff: 1s, 2s, 4s between retries
        - Requires AnkiWeb credentials to be configured
    """
    from suomi.anki import call
    
    for attempt in range(max_retries):
        try:
            # TODO: Investigate if AnkiConnect's sync action requires
            # pre-configured credentials or if we can pass them as parameters
            result = call("sync")
            print(f"Sync successful: {result}")
            return
        except Exception as e:
            if attempt == max_retries - 1:
                raise SyncError(
                    f"Sync failed after {max_retries} attempts. "
                    f"Last error: {e}"
                ) from e
            
            wait_time = 2 ** attempt
            print(f"Sync attempt {attempt + 1} failed, retrying in {wait_time}s...")
            time.sleep(wait_time)

## High-Level API

In [ ]:
#| export
def addnotes_with_sync(
    deck: str,  # Anki deck name
    tsv: str    # Path to TSV file
) -> None:
    """Add notes from TSV and sync to AnkiWeb automatically.
    
    This is a convenience wrapper around suomi.anki.addnotes() that
    automatically syncs to AnkiWeb after adding cards.
    
    Args:
        deck: Anki deck name
        tsv: Path to TSV file with Finnish vocabulary
    
    Raises:
        SyncError: If sync fails after card addition
    
    Notes:
        Cards are added successfully even if sync fails.
        You can manually retry sync using sync_to_ankiweb().
    """
    from suomi.anki import addnotes
    
    # Add cards to Anki (via AnkiConnect)
    addnotes(deck, tsv)
    
    # Sync to AnkiWeb
    print(f"Cards added to deck '{deck}', syncing to AnkiWeb...")
    sync_to_ankiweb()

## EDA / Testing

**Note**: These cells are for exploration and testing. Set `eval: false` to prevent execution during `nbdev_test`.

In [ ]:
#| eval: false
# Test configuration (will fail if env vars not set)
try:
    config = configure_ankiweb()
    print(f"Configuration loaded: {config}")
except ValueError as e:
    print(f"Expected error: {e}")

In [ ]:
#| eval: false
# Test sync (requires Anki + AnkiConnect running)
try:
    sync_to_ankiweb()
except Exception as e:
    print(f"Sync test: {e}")

In [ ]:
#| eval: false
# Test addnotes_with_sync (requires test TSV file)
# addnotes_with_sync("TestDeck", "test.tsv")

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()